In [25]:
# Reload modules automatically
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import logging
import math

import numpy as np
import torch
from strn_and_rbstness.attacks import Attack, create_attack
from strn_and_rbstness.data import GraphDataset, split
from strn_and_rbstness.helper.utils import accuracy, count_edges
from strn_and_rbstness.models import create_model
from strn_and_rbstness.train import _train
from common import CSBM

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Sample CSBM

In [22]:
seed = 0
n = 1000
avg_intra_degree = 1.5 * 2 # intra_edges_per_node * 2
avg_inter_degree = 0.5 * 2
p = avg_intra_degree * 2 / (n - 1)
q = avg_inter_degree * 2 / (n - 1)
K = 0.5 # Defines distance between means of the gaußians in sigma-units
sigma = 0.1
d = round(n / math.log(n)**2)
mu = np.array([K*sigma / (2 * d**0.5) for i in range(d)], dtype=np.float32)
cov = sigma**2 * np.identity(d, dtype=np.float32)

# X, A ~ CSBM(n, p, q, mu, cov)
csbm = CSBM(p, q, mu, cov)
X, A, y = csbm.sample(n, seed=0)
#csbm.check_separabilities(X, A, y)
print(f"Dim: {d}")

Dim: 21


In [24]:
i = 1
pot_neighbours_different = np.arange(0, n)[y != y[i]]
pot_neighbours_same = np.arange(0, n)[y == y[i]]
X_diff = X[i,:] - X[pot_neighbours_different,:]
X_diff_same = X[i,:] - X[pot_neighbours_same,:]
X_dist = np.linalg.norm(X_diff, ord=2, axis=1)
X_dist_same = np.linalg.norm(X_diff_same, ord=2, axis=1)
print(X_diff.shape)
print(X_dist.shape)
print(np.flip(np.sort(X_dist))[:10])
print(np.flip(np.argsort(X_dist))[:10])
print(X_dist[np.flip(np.argsort(X_dist))[:10]])
#print(np.flip(np.sort(X_dist_same))[:100])

(496, 21)
(496,)
[0.89766749 0.88119925 0.87545127 0.86959216 0.84987773 0.84460017
 0.84241947 0.84066322 0.83618692 0.81684797]
[466 194 227 241 366 302 391  24 469  64]
[0.89766749 0.88119925 0.87545127 0.86959216 0.84987773 0.84460017
 0.84241947 0.84066322 0.83618692 0.81684797]


## Configure GNN & Attack

In [103]:
"""
model_params = dict(
    label="GCN",
    model="GCN", #GCN or DenseGCN
    n_filters=64,
    dropout=0.5
)
"""
model_params = dict(
    label="APPNP",
    model="APPNP", #GCN or DenseGCN
    n_hidden=64,
    dropout=0.,
    K=10,
    alpha=0.1
)
"""
train_params = dict(
    loss_type="CE",
    lr=1e-2,
    weight_decay=1e-3,
    patience=300,
    max_epochs=3000,
    use_selftrain = False, 
    use_advtrain = False,
)
"""
train_params = dict(
    loss_type="CE",
    lr=1e-2,
    weight_decay=1e-3,
    patience=300,
    max_epochs=1000,
    use_selftrain = False, 
    use_advtrain = False,
)
"""
attack = "PGD"
attack_params = dict(
    epochs=200,
    base_lr=1e-2,
    scale_lr_with_n_attacked_edges=True,
    loss_type="tanhMargin" # CE or tanhMargin or CW
)
"""
attack = 'PRBCD'
attack_params = dict(
    epochs=500,
    fine_tune_epochs=100,
    keep_heuristic="WeightOnly",
    search_space_size=1_000,
    do_synchronize=True,
    loss_type="tanhMargin"
)
epsilon = 0.05

# Other
split_params = {
    "strategy": "normal", # or "custom"
    "p_trn": 1,
    "p_tst": 0, # "normal" uses 1 - p_trn, only for custom split strategy
    "p_selftrn": 0 # Refers to unlabeled data, which is not test data, 
                    # only for custom split strategy
}
verbosity_params = dict(
    display_steps = 100
)   
# Device
device = 0
if not torch.cuda.is_available():
    device == "cpu", "CUDA is not availble, set device to 'cpu'"
else:
    device = torch.device(f"cuda:{device}")
    logging.info(f"Currently on gpu device {device}")
attack_params["data_device"] = device

INFO:root:Currently on gpu device cuda:0


#### Train GNN

In [104]:
torch.manual_seed(seed)
np.random.seed(seed)
split_ids = split(y, split_params, seed)
X_gpu = torch.tensor(X, dtype=torch.float32, device=device)
A_gpu = torch.tensor(A, dtype=torch.float32, device=device)
y_gpu = torch.tensor(y, device=device)
graph = GraphDataset((X_gpu, A_gpu, y_gpu), split_ids)
model_params = dict(**model_params, 
                    n_features=graph.get_n_features(), 
                    n_classes=graph.get_n_classes())
model = create_model(model_params).to(device)
statistics = _train(model, graph, train_params, verbosity_params, None)
#best_epoch = np.argmin(statistics[1])

INFO:root:
Epoch    0: loss_train: 0.69337, loss_val: 0.69340, acc_train: 0.50450, acc_val: 0.48452
INFO:root:
Epoch  100: loss_train: 0.61028, loss_val: 0.63504, acc_train: 0.68268, acc_val: 0.66034
INFO:root:
Epoch  200: loss_train: 0.55015, loss_val: 0.62300, acc_train: 0.75275, acc_val: 0.67233
INFO:root:
Epoch  300: loss_train: 0.53251, loss_val: 0.61919, acc_train: 0.76777, acc_val: 0.68232
INFO:root:
Epoch  400: loss_train: 0.52566, loss_val: 0.61773, acc_train: 0.77377, acc_val: 0.68132
INFO:root:
Epoch  500: loss_train: 0.52176, loss_val: 0.61517, acc_train: 0.78278, acc_val: 0.68132
INFO:root:
Epoch  600: loss_train: 0.51829, loss_val: 0.61405, acc_train: 0.78979, acc_val: 0.67333
INFO:root:
Epoch  700: loss_train: 0.51706, loss_val: 0.61308, acc_train: 0.78979, acc_val: 0.67732
INFO:root:
Epoch  800: loss_train: 0.51657, loss_val: 0.61247, acc_train: 0.78679, acc_val: 0.67632
INFO:root:
Epoch  900: loss_train: 0.51607, loss_val: 0.61214, acc_train: 0.78879, acc_val: 0.67632


In [98]:
idx_trn, idx_val, _, _ = split_ids
idx_all = np.arange(len(y))
model.eval()
logits = model(X_gpu, A_gpu)
acc_trn = accuracy(logits[idx_trn], y_gpu[idx_trn])
acc_val = accuracy(logits[idx_val], y_gpu[idx_val])
print(acc_trn)
print(acc_val)

0.7957957983016968
0.687312662601471


#### Attack GNN

In [105]:
idx_trn, _, _, _ = split_ids
adversary = create_attack(attack, attr=X_gpu, adj=A_gpu, labels=y_gpu, 
                            model=model, idx_attack=idx_trn, device=device, 
                            binary_attr=False,
                            make_undirected=True, 
                            **attack_params)
m  = count_edges(A_gpu, idx_trn)
n_perturbations = int(round(epsilon * m))
print(f"#Edges: {m} -> budget: {n_perturbations}")
adversary.attack(n_perturbations, _run=None)
A_pert, X_pert = adversary.get_pertubations()
logits, acc = Attack.evaluate_global(model, X_pert, A_pert, y_gpu, idx_trn)
print(f"Accuracy: {acc}")

INFO:root:
Before the attack - Loss: -0.3784329891204834 Accuracy: 78.879 %



#Edges: 2064 -> budget: 103


  0%|          | 0/500 [00:00<?, ?it/s]INFO:root:
Epoch: 0 Loss: -0.3784329891204834 Accuracy: 78.579 %

  4%|▍         | 20/500 [00:00<00:11, 42.01it/s]INFO:root:
Epoch: 20 Loss: -0.3049657344818115 Accuracy: 74.274 %

  8%|▊         | 38/500 [00:00<00:12, 38.14it/s]INFO:root:
Epoch: 40 Loss: -0.29116836190223694 Accuracy: 72.973 %

 12%|█▏        | 58/500 [00:01<00:11, 37.60it/s]INFO:root:
Epoch: 60 Loss: -0.2788376808166504 Accuracy: 72.272 %

 16%|█▌        | 78/500 [00:02<00:11, 37.26it/s]INFO:root:
Epoch: 80 Loss: -0.27123579382896423 Accuracy: 71.471 %

 20%|█▉        | 98/500 [00:02<00:10, 36.87it/s]INFO:root:
Epoch: 100 Loss: -0.26667264103889465 Accuracy: 71.972 %

 24%|██▎       | 118/500 [00:03<00:10, 36.77it/s]INFO:root:
Epoch: 120 Loss: -0.2635248899459839 Accuracy: 71.271 %

 28%|██▊       | 138/500 [00:03<00:09, 36.37it/s]INFO:root:
Epoch: 140 Loss: -0.2615368962287903 Accuracy: 70.971 %

 32%|███▏      | 158/500 [00:04<00:09, 36.38it/s]INFO:root:
Epoch: 160 Loss: -0.26

Accuracy: 0.7217217087745667


In [106]:
logits = model(X_gpu, A_gpu)
correct = (logits.argmax(1) == y_gpu)[idx_trn]
logits_pert = model(X_pert, A_pert)
correct_pert = (logits_pert.argmax(1) == y_gpu)[idx_trn]

In [107]:
print(f"Correctly Classified: {sum(correct)}")
print(f"Correctly Classified Pert: {sum(correct_pert)}")
correct_after_pert = correct[correct] == correct_pert[correct]
print(f"Still correct after perturbation: {sum(correct_after_pert)}")
flipped = correct[correct] != correct_pert[correct]
print(f"Flipped Predictions: {sum(flipped)}")

Correctly Classified: 788
Correctly Classified Pert: 721
Still correct after perturbation: 717
Flipped Predictions: 71


#### Eval Separability

In [108]:
ids = np.arange(n)[idx_trn][correct.cpu().numpy()][flipped.cpu().numpy()]
n_corr, n_wrong = csbm.likelihood_separability(X, A, y, ids)
n_corr_pert, n_wrong_pert = csbm.likelihood_separability(X, A_pert.to_dense().cpu().numpy(), y, ids)
p_not_adv = (n_wrong_pert - n_wrong) / sum(flipped)
print(f"Percentage of flipped predictions not adversarial: {p_not_adv}")

Percentage of flipped predictions not adversarial: 0.07042253017425537


In [70]:
ids = np.arange(n)[idx_trn][correct.cpu().numpy()][flipped.cpu().numpy()]
csbm.check_separabilities(X, A, y, ids)
csbm.check_separabilities(X, A_pert.to_dense().cpu().numpy(), y, ids)

Feature Separability:
n_corr: 15
n_wrong: 27
Structure Separability:
n_corr: 37
n_wrong: 5
Likelihood Separability:
n_corr: 38
n_wrong: 4
Feature Separability:
n_corr: 15
n_wrong: 27
Structure Separability:
n_corr: 33
n_wrong: 9
Likelihood Separability:
n_corr: 33
n_wrong: 9


# Experiment 3 seeds, APPNP, PR-PGD

In [34]:
def get_sbm_model(n):
    avg_intra_degree = 1.5 * 2 # intra_edges_per_node * 2
    avg_inter_degree = 0.5 * 2
    p = avg_intra_degree * 2 / (n - 1)
    q = avg_inter_degree * 2 / (n - 1)
    K = 0.5 # Defines distance between means of the gaußians in sigma-units
    sigma = 0.1
    d = round(n / math.log(n)**2)
    mu = np.array([K*sigma / (2 * d**0.5) for i in range(d)], dtype=np.float32)
    cov = sigma**2 * np.identity(d, dtype=np.float32)
    return CSBM(p, q, mu, cov)

def evaluate(likelihood_change_l, bayes_acc_trn_l, bayes_acc_val_l, 
             gnn_acc_trn_l, gnn_acc_val_l, flipped_l, flipped_not_adv_l):
    likelihood_change_mean = np.mean(likelihood_change_l) * 100
    likelihood_change_std = np.std(likelihood_change_l) * 100
    bayes_acc_trn_mean = np.mean(bayes_acc_trn_l) * 100 
    bayes_acc_trn_std = np.std(bayes_acc_trn_l) * 100
    bayes_acc_val_mean = np.mean(bayes_acc_val_l) * 100 
    bayes_acc_val_std = np.std(bayes_acc_val_l) * 100
    gnn_acc_trn_mean = np.mean(gnn_acc_trn_l) * 100
    gnn_acc_trn_std = np.std(gnn_acc_trn_l) * 100
    gnn_acc_val_mean = np.mean(gnn_acc_val_l) * 100
    gnn_acc_val_std = np.std(gnn_acc_val_l) * 100
    flipped_mean = np.mean(flipped_l)
    flipped_std = np.std(flipped_l)
    flipped_not_adv_mean = np.mean(flipped_not_adv_l)
    flipped_not_adv_std = np.std(flipped_not_adv_l)
    print(f"Avg. Bayes Classification Error Trn: {bayes_acc_trn_mean:.2f}% +- {bayes_acc_trn_std:.2f}%")
    print(f"Avg. Bayes Classification Error Val: {bayes_acc_val_mean:.2f}% +- {bayes_acc_val_std:.2f}%")
    print(f"Avg. GNN Classification Error Trn: {gnn_acc_trn_mean:.2f}% +- {gnn_acc_trn_std:.2f}%")
    print(f"Avg. GNN Classification Error Val: {gnn_acc_val_mean:.2f}% +- {gnn_acc_val_std:.2f}%")
    print(f"Avg. Flipped Nodes: {flipped_mean:.1f} +- {flipped_std:.1f}")
    print(f"Avg. Flipped Nodes violating BC-Criterion: {flipped_not_adv_mean:.1f} +- {flipped_not_adv_std:.1f}")
    print(f"Avg. Flipped Nodes violating BC-Criterion: {likelihood_change_mean:.2f}% +- {likelihood_change_std:.2f}%")

def perform_experiment(n_s, seeds):
    for n in n_s:
        budget = -1
        flipped_l = []
        flipped_not_adv_l = []
        likelihood_change_l = []
        bayes_acc_l = []
        bayes_acc_trn_l = []
        bayes_acc_val_l = []
        gnn_acc_trn_l = []
        gnn_acc_val_l = []
        for seed in seeds:
            torch.manual_seed(seed)
            np.random.seed(seed)

            # X, A ~ CSBM(n, p, q, mu, cov)
            csbm = get_sbm_model(n)
            X, A, y = csbm.sample(n, seed)

            # Train
            split_ids = split(y, split_params, seed)
            X_gpu = torch.tensor(X, dtype=torch.float32, device=device)
            A_gpu = torch.tensor(A, dtype=torch.float32, device=device)
            y_gpu = torch.tensor(y, device=device)
            graph = GraphDataset((X_gpu, A_gpu, y_gpu), split_ids)
            model_params_trn = dict(**model_params, 
                                    n_features=graph.get_n_features(), 
                                    n_classes=graph.get_n_classes())
            model = create_model(model_params_trn).to(device)
            statistics = _train(model, graph, train_params, verbosity_params, None)
            
            # Attack
            idx_trn, idx_val, _, _ = split_ids
            adversary = create_attack(attack, attr=X_gpu, adj=A_gpu, labels=y_gpu, 
                                    model=model, idx_attack=idx_trn, device=device, 
                                    binary_attr=False,
                                    make_undirected=True, 
                                    display_step=301,
                                    **attack_params)
            m  = count_edges(A_gpu, idx_trn)
            n_perturbations = int(round(epsilon * m))
            budget = n_perturbations
            print(f"#Edges: {m} -> budget: {n_perturbations}")
            adversary.attack(n_perturbations, _run=None)
            A_pert, X_pert = adversary.get_pertubations()
            logits, acc = Attack.evaluate_global(model, X_pert, A_pert, y_gpu, idx_trn)
            print(f"Accuracy Attack (trn): {acc}")

            # Eval
            # Bayes Classifier:
            n_corr, n_wrong = csbm.likelihood_separability(X, A, y)
            bayes_acc_l.append(n_corr / n)
            n_corr, n_wrong = csbm.likelihood_separability(X, A, y, ids=idx_trn)
            bayes_acc_trn_l.append(2 * n_corr / n )
            n_corr, n_wrong = csbm.likelihood_separability(X, A, y, ids=idx_val)
            bayes_acc_val_l.append(2 * n_corr / n)
            # GNN Accuracies
            model.eval()
            logits = model(X_gpu, A_gpu)
            acc_trn = accuracy(logits[idx_trn], y_gpu[idx_trn])
            acc_val = accuracy(logits[idx_val], y_gpu[idx_val])
            gnn_acc_trn_l.append(acc_trn)
            gnn_acc_val_l.append(acc_val)
            print(f"Accuracy model (trn): {acc_trn}, (val): {acc_val}")
            # Count Flips violating Bayes Criterion
            correct = (logits.argmax(1) == y_gpu)[idx_trn]
            logits_pert = model(X_pert, A_pert)
            correct_pert = (logits_pert.argmax(1) == y_gpu)[idx_trn]
            flipped = correct[correct] != correct_pert[correct]

            ids = np.arange(n)[idx_trn][correct.cpu().numpy()][flipped.cpu().numpy()]
            ids_bayes_separable = []
            for i in ids:
                bayes_separable, _ = csbm.likelihood_separability(X, A, y, [i])
                if bayes_separable:
                    ids_bayes_separable.append(i)
            n_corr_pert, n_wrong_pert = csbm.likelihood_separability(X, A_pert.to_dense().cpu().numpy(), y, ids_bayes_separable)
            p_adv = n_corr_pert / sum(flipped).cpu()
            p_not_adv = 1 - p_adv
            flipped_l.append(sum(flipped).cpu())
            flipped_not_adv_l.append(sum(flipped).cpu() - n_corr_pert)
            likelihood_change_l.append(p_not_adv)
        print(f"Evaluting experiment with n={n}:")
        print(f"Note: #Edges: {m} -> budget: {n_perturbations}")
        evaluate(likelihood_change_l, bayes_acc_trn_l, bayes_acc_val_l, 
                gnn_acc_trn_l, gnn_acc_val_l, flipped_l, flipped_not_adv_l)
        print(f"Likelihood_change_l: {likelihood_change_l}")

In [35]:
model_params = dict(
    label="APPNP",
    model="APPNP", #GCN or DenseGCN
    n_hidden=64,
    dropout=0.,
    K=10,
    alpha=0.1
)
train_params = dict(
    loss_type="CE",
    lr=1e-2,
    weight_decay=1e-3,
    patience=300,
    max_epochs=1000,
    use_selftrain = False, 
    use_advtrain = False,
)
attack = 'PRBCD'
attack_params = dict(
    epochs=300,
    fine_tune_epochs=100,
    keep_heuristic="WeightOnly",
    search_space_size=1_000,
    do_synchronize=True,
    loss_type="tanhMargin"
)

# Other
split_params = {
    "strategy": "normal", # or "custom"
    "p_trn": 1,
    "p_tst": 0, # "normal" uses 1 - p_trn, only for custom split strategy
    "p_selftrn": 0 # Refers to unlabeled data, which is not test data, 
                    # only for custom split strategy
}
verbosity_params = dict(
    display_steps = 1001
)   
# Device
device = 0
if not torch.cuda.is_available():
    device == "cpu", "CUDA is not availble, set device to 'cpu'"
else:
    device = torch.device(f"cuda:{device}")
    logging.info(f"Currently on gpu device {device}")
attack_params["data_device"] = device

INFO:root:Currently on gpu device cuda:0


In [36]:
epsilon = 0.05
seeds = [0, 1, 2, 3, 4]
n_s = [1000]
perform_experiment(n_s, seeds)


INFO:root:
Epoch    0: loss_train: 0.69438, loss_val: 0.69506, acc_train: 0.50400, acc_val: 0.50400
INFO:root:
Epoch  937: loss_train: 0.49360, loss_val: 0.57912, acc_train: 0.77800, acc_val: 0.70800
INFO:root:
Before the attack - Loss: -0.4128348231315613 Accuracy: 77.000 %



#Edges: 1032 -> budget: 52


  0%|          | 0/300 [00:00<?, ?it/s]INFO:root:
Epoch: 0 Loss: -0.4128347933292389 Accuracy: 76.600 %

 66%|██████▌   | 197/300 [00:04<00:02, 46.11it/s]INFO:root:Loading search space of epoch 119 (accuarcy=0.6580000519752502) for fine tuning

100%|██████████| 300/300 [00:06<00:00, 46.21it/s]
INFO:root:2-th sampling: too many samples 58.0
INFO:root:6-th sampling: too many samples 54.0
INFO:root:7-th sampling: too many samples 57.0
INFO:root:8-th sampling: too many samples 55.0
INFO:root:10-th sampling: too many samples 53.0
INFO:root:12-th sampling: too many samples 58.0
INFO:root:14-th sampling: too many samples 58.0
INFO:root:19-th sampling: too many samples 55.0


Accuracy Attack (trn): 0.6980000138282776
Accuracy model (trn): 0.7700000405311584, (val): 0.7100000381469727


INFO:root:
Epoch    0: loss_train: 0.69518, loss_val: 0.69518, acc_train: 0.47800, acc_val: 0.47800
INFO:root:
Epoch  971: loss_train: 0.47722, loss_val: 0.61559, acc_train: 0.79600, acc_val: 0.69400
INFO:root:
Before the attack - Loss: -0.4314100444316864 Accuracy: 79.400 %



...

Accuracy Attack (trn): 0.7239999771118164
Accuracy model (trn): 0.8140000104904175, (val): 0.7400000095367432


INFO:root:
Epoch    0: loss_train: 0.69416, loss_val: 0.69448, acc_train: 0.51000, acc_val: 0.51000
INFO:root:
Epoch  818: loss_train: 0.49713, loss_val: 0.59676, acc_train: 0.80000, acc_val: 0.69400
INFO:root:
Before the attack - Loss: -0.4090399444103241 Accuracy: 81.000 %



#Edges: 998 -> budget: 50


  0%|          | 0/300 [00:00<?, ?it/s]INFO:root:
Epoch: 0 Loss: -0.4090399444103241 Accuracy: 79.600 %

 66%|██████▌   | 197/300 [00:04<00:02, 45.32it/s]INFO:root:Loading search space of epoch 142 (accuarcy=0.7020000219345093) for fine tuning

100%|██████████| 300/300 [00:06<00:00, 46.05it/s]
INFO:root:1-th sampling: too many samples 59.0
INFO:root:2-th sampling: too many samples 62.0
INFO:root:3-th sampling: too many samples 57.0
INFO:root:7-th sampling: too many samples 57.0
INFO:root:10-th sampling: too many samples 58.0
INFO:root:11-th sampling: too many samples 64.0
INFO:root:12-th sampling: too many samples 54.0
INFO:root:15-th sampling: too many samples 51.0
INFO:root:17-th sampling: too many samples 54.0
INFO:root:18-th sampling: too many samples 52.0


Accuracy Attack (trn): 0.7200000286102295
Accuracy model (trn): 0.8100000619888306, (val): 0.7040000557899475
Evaluting experiment with n=1000:
Note: #Edges: 998 -> budget: 50
Avg. Bayes Classification Error Trn: 85.80% +- 2.40%
Avg. Bayes Classification Error Val: 85.48% +- 1.38%
Avg. GNN Classification Error Trn: 79.24% +- 1.80%
Avg. GNN Classification Error Val: 71.52% +- 1.75%
Avg. Flipped Nodes: 43.6 +- 4.0
Avg. Flipped Nodes violating BC-Criterion: 11.0 +- 2.3
Avg. Flipped Nodes violating BC-Criterion: 25.34% +- 5.44%
Likelihood_change_l: [tensor(0.3421), tensor(0.2000), tensor(0.2000), tensor(0.2391), tensor(0.2857)]


In [144]:
# old
epsilon = 0.05
seeds = [0, 1, 2, 3, 4]
n_s = [1000]
perform_experiment(n_s, seeds)


INFO:root:
Epoch    0: loss_train: 0.69438, loss_val: 0.69506, acc_train: 0.50400, acc_val: 0.50400
INFO:root:
Epoch  956: loss_train: 0.49914, loss_val: 0.58164, acc_train: 0.78000, acc_val: 0.71200
/nfs/homedirs/gosl/src/selftraining-and-robustness/strn_and_rbstness/attacks/base_attack.py:77: UserWarning: The attack will fail if the gradient w.r.t. the adjacency can't be computed.
  warnings.warn("The attack will fail if the gradient w.r.t. the adjacency can't be computed.")
/nfs/homedirs/gosl/src/selftraining-and-robustness/strn_and_rbstness/attacks/prbcd.py:392: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  + (n - row_idx) * ((n - row_idx) - 1) // 2
INFO:r

#Edges: 1032 -> budget: 52


  0%|          | 0/300 [00:00<?, ?it/s]INFO:root:
Epoch: 0 Loss: -0.40562307834625244 Accuracy: 77.200 %

 66%|██████▌   | 198/300 [00:05<00:02, 35.39it/s]INFO:root:Loading search space of epoch 109 (accuarcy=0.6580000519752502) for fine tuning

100%|██████████| 300/300 [00:08<00:00, 35.47it/s]
INFO:root:1-th sampling: too many samples 53.0
INFO:root:4-th sampling: too many samples 53.0
INFO:root:6-th sampling: too many samples 55.0
INFO:root:7-th sampling: too many samples 65.0
INFO:root:14-th sampling: too many samples 56.0
INFO:root:15-th sampling: too many samples 56.0
INFO:root:16-th sampling: too many samples 56.0
INFO:root:18-th sampling: too many samples 53.0
INFO:root:19-th sampling: too many samples 56.0


Accuracy Attack (trn): 0.6980000138282776
Accuracy model (trn): 0.7800000309944153, (val): 0.7120000123977661


INFO:root:
Epoch    0: loss_train: 0.69518, loss_val: 0.69518, acc_train: 0.47800, acc_val: 0.47800
INFO:root:
Epoch  990: loss_train: 0.47481, loss_val: 0.61701, acc_train: 0.79200, acc_val: 0.67800
INFO:root:
Before the attack - Loss: -0.4358418881893158 Accuracy: 79.200 %



...

Accuracy Attack (trn): 0.722000002861023
Accuracy model (trn): 0.8020000457763672, (val): 0.7300000190734863


INFO:root:
Epoch    0: loss_train: 0.69416, loss_val: 0.69448, acc_train: 0.51000, acc_val: 0.51000
INFO:root:
Epoch  927: loss_train: 0.49677, loss_val: 0.59835, acc_train: 0.80200, acc_val: 0.69400
INFO:root:
Before the attack - Loss: -0.4103357195854187 Accuracy: 81.200 %



#Edges: 998 -> budget: 50


  0%|          | 0/300 [00:00<?, ?it/s]INFO:root:
Epoch: 0 Loss: -0.4103356599807739 Accuracy: 80.600 %

 66%|██████▌   | 197/300 [00:04<00:02, 40.67it/s]INFO:root:Loading search space of epoch 175 (accuarcy=0.6980000138282776) for fine tuning

100%|██████████| 300/300 [00:07<00:00, 42.50it/s]
INFO:root:1-th sampling: too many samples 59.0
INFO:root:2-th sampling: too many samples 53.0
INFO:root:3-th sampling: too many samples 52.0
INFO:root:6-th sampling: too many samples 58.0
INFO:root:7-th sampling: too many samples 53.0
INFO:root:12-th sampling: too many samples 58.0
INFO:root:13-th sampling: too many samples 54.0
INFO:root:17-th sampling: too many samples 51.0
INFO:root:18-th sampling: too many samples 51.0
INFO:root:19-th sampling: too many samples 51.0


Accuracy Attack (trn): 0.7200000286102295
Accuracy model (trn): 0.812000036239624, (val): 0.690000057220459
Evaluting experiment with n=1000:
Note: #Edges: 998 -> budget: 50
Avg. Bayes Classification Error Trn: 85.80% +- 2.40%
Avg. Bayes Classification Error Val: 85.48% +- 1.38%
Avg. GNN Classification Error Trn: 79.20% +- 1.39%
Avg. GNN Classification Error Val: 70.88% +- 1.99%
Avg. Flipped Nodes: 45.4 +- 2.7
Avg. Flipped Nodes violating BC-Criterion: 6.6 +- 1.7
Avg. Flipped Nodes violating BC-Criterion: 14.76% +- 4.85%
Likelihood_change_l: [tensor(0.1136), tensor(0.1250), tensor(0.1304), tensor(0.2439), tensor(0.1250)]


In [37]:
epsilon = 0.1
seeds = [0, 1, 2, 3, 4]
n_s = [1000]
perform_experiment(n_s, seeds)


INFO:root:
Epoch    0: loss_train: 0.69438, loss_val: 0.69506, acc_train: 0.50400, acc_val: 0.50400
INFO:root:
Epoch  999: loss_train: 0.50672, loss_val: 0.57944, acc_train: 0.75800, acc_val: 0.72600
INFO:root:
Before the attack - Loss: -0.3909813463687897 Accuracy: 75.600 %



#Edges: 1032 -> budget: 103


  0%|          | 0/300 [00:00<?, ?it/s]INFO:root:
Epoch: 0 Loss: -0.3909812867641449 Accuracy: 73.800 %

 66%|██████▌   | 197/300 [00:04<00:02, 47.02it/s]INFO:root:Loading search space of epoch 192 (accuarcy=0.5960000157356262) for fine tuning

100%|██████████| 300/300 [00:06<00:00, 47.24it/s]
INFO:root:2-th sampling: too many samples 111.0
INFO:root:5-th sampling: too many samples 109.0
INFO:root:7-th sampling: too many samples 106.0
INFO:root:11-th sampling: too many samples 122.0
INFO:root:15-th sampling: too many samples 109.0
INFO:root:16-th sampling: too many samples 108.0
INFO:root:17-th sampling: too many samples 111.0


Accuracy Attack (trn): 0.6340000033378601
Accuracy model (trn): 0.7560000419616699, (val): 0.7160000205039978


INFO:root:
Epoch    0: loss_train: 0.69518, loss_val: 0.69518, acc_train: 0.47800, acc_val: 0.47800
INFO:root:
Epoch  442: loss_train: 0.49237, loss_val: 0.62749, acc_train: 0.77000, acc_val: 0.67600
INFO:root:
Before the attack - Loss: -0.4070720672607422 Accuracy: 77.800 %



...

Accuracy Attack (trn): 0.6539999842643738
Accuracy model (trn): 0.8020000457763672, (val): 0.7240000367164612


INFO:root:
Epoch    0: loss_train: 0.69416, loss_val: 0.69448, acc_train: 0.51000, acc_val: 0.51000
INFO:root:
Epoch  626: loss_train: 0.49968, loss_val: 0.59767, acc_train: 0.80200, acc_val: 0.69600
INFO:root:
Before the attack - Loss: -0.4056304395198822 Accuracy: 80.400 %



#Edges: 998 -> budget: 100


  0%|          | 0/300 [00:00<?, ?it/s]INFO:root:
Epoch: 0 Loss: -0.4056304395198822 Accuracy: 79.000 %

 66%|██████▌   | 197/300 [00:04<00:02, 45.84it/s]INFO:root:Loading search space of epoch 181 (accuarcy=0.6480000019073486) for fine tuning

100%|██████████| 300/300 [00:06<00:00, 46.74it/s]
INFO:root:1-th sampling: too many samples 112.0
INFO:root:2-th sampling: too many samples 112.0
INFO:root:4-th sampling: too many samples 108.0
INFO:root:5-th sampling: too many samples 103.0
INFO:root:7-th sampling: too many samples 111.0
INFO:root:10-th sampling: too many samples 104.0
INFO:root:11-th sampling: too many samples 101.0
INFO:root:15-th sampling: too many samples 104.0
INFO:root:16-th sampling: too many samples 111.0
INFO:root:17-th sampling: too many samples 115.0
INFO:root:18-th sampling: too many samples 105.0
INFO:root:19-th sampling: too many samples 107.0


Accuracy Attack (trn): 0.6759999990463257
Accuracy model (trn): 0.8040000200271606, (val): 0.6960000395774841
Evaluting experiment with n=1000:
Note: #Edges: 998 -> budget: 100
Avg. Bayes Classification Error Trn: 85.80% +- 2.40%
Avg. Bayes Classification Error Val: 85.48% +- 1.38%
Avg. GNN Classification Error Trn: 78.24% +- 1.83%
Avg. GNN Classification Error Val: 70.92% +- 1.86%
Avg. Flipped Nodes: 70.0 +- 4.8
Avg. Flipped Nodes violating BC-Criterion: 15.4 +- 3.0
Avg. Flipped Nodes violating BC-Criterion: 21.94% +- 3.55%
Likelihood_change_l: [tensor(0.2381), tensor(0.1972), tensor(0.2000), tensor(0.2800), tensor(0.1818)]


In [130]:
# Results over all 5 seeds:
evaluate(likelihood_change_l, bayes_acc_trn_l, bayes_acc_val_l, 
            gnn_acc_trn_l, gnn_acc_val_l, flipped_l, flipped_not_adv_l)

Avg. Bayes Classification Error Trn: 85.80% +- 2.40%
Avg. Bayes Classification Error Val: 85.48% +- 1.38%
Avg. GNN Classification Error Trn: 79.16% +- 1.48%
Avg. GNN Classification Error Val: 71.08% +- 2.14%
Avg. Flipped Nodes: 43.4 +- 1.2
Avg. Flipped Nodes violating BC-Criterion: 7.2 +- 2.6
Avg. Flipped Nodes violating BC-Criterion: 16.70% +- 6.22%


### GCN with PGD

In [38]:
model_params = dict(
    label="GCN",
    model="DenseGCN", #GCN or DenseGCN
    n_filters=64,
    dropout=0.5
)
train_params = dict(
    loss_type="CE",
    lr=1e-2,
    weight_decay=1e-3,
    patience=300,
    max_epochs=1000,
    use_selftrain = False, 
    use_advtrain = False,
)
attack = "PGD"
attack_params = dict(
    epochs=200,
    base_lr=1e-2,
    scale_lr_with_n_attacked_edges=True,
    loss_type="tanhMargin" # CE or tanhMargin or CW
)
epsilon = 0.05

# Other
split_params = {
    "strategy": "normal", # or "custom"
    "p_trn": 1,
    "p_tst": 0, # "normal" uses 1 - p_trn, only for custom split strategy
    "p_selftrn": 0 # Refers to unlabeled data, which is not test data, 
                    # only for custom split strategy
}
verbosity_params = dict(
    display_steps = 1001
)   
# Device
device = 0
if not torch.cuda.is_available():
    device == "cpu", "CUDA is not availble, set device to 'cpu'"
else:
    device = torch.device(f"cuda:{device}")
    logging.info(f"Currently on gpu device {device}")
attack_params["data_device"] = device

INFO:root:Currently on gpu device cuda:0


In [39]:
seeds = [0, 1, 2, 3, 4]
n_s = [140, 500, 1000]
perform_experiment(n_s, seeds)

INFO:root:
Epoch    0: loss_train: 0.69189, loss_val: 0.69158, acc_train: 0.55072, acc_val: 0.56338
INFO:root:
Epoch  919: loss_train: 0.45602, loss_val: 0.52335, acc_train: 0.76812, acc_val: 0.76056


#Edges: 128 -> budget: 6


INFO:root:
Epoch    0: loss_train: 0.69228, loss_val: 0.69239, acc_train: 0.63768, acc_val: 0.59155


Accuracy Attack (trn): 0.7536231875419617
Accuracy model (trn): 0.8115942478179932, (val): 0.7323943376541138


INFO:root:
Epoch  227: loss_train: 0.48165, loss_val: 0.53197, acc_train: 0.76812, acc_val: 0.71831


...

#Edges: 1045 -> budget: 52
Accuracy Attack (trn): 0.7300000190734863
Accuracy model (trn): 0.8380000591278076, (val): 0.734000027179718


INFO:root:
Epoch    0: loss_train: 0.69274, loss_val: 0.69284, acc_train: 0.54000, acc_val: 0.53800
INFO:root:
Epoch  954: loss_train: 0.47423, loss_val: 0.54607, acc_train: 0.78800, acc_val: 0.73400


#Edges: 1017 -> budget: 51
Accuracy Attack (trn): 0.6859999895095825
Accuracy model (trn): 0.796000063419342, (val): 0.7300000190734863


INFO:root:
Epoch    0: loss_train: 0.69290, loss_val: 0.69320, acc_train: 0.49000, acc_val: 0.48000
INFO:root:
Epoch  347: loss_train: 0.50505, loss_val: 0.59935, acc_train: 0.76800, acc_val: 0.68800


#Edges: 998 -> budget: 50
Accuracy Attack (trn): 0.7139999866485596
Accuracy model (trn): 0.800000011920929, (val): 0.6840000152587891
Evaluting experiment with n=1000:
Note: #Edges: 998 -> budget: 50
Avg. Bayes Classification Error Trn: 85.80% +- 2.40%
Avg. Bayes Classification Error Val: 85.48% +- 1.38%
Avg. GNN Classification Error Trn: 80.52% +- 1.72%
Avg. GNN Classification Error Val: 70.52% +- 2.51%
Avg. Flipped Nodes: 51.6 +- 4.6
Avg. Flipped Nodes violating BC-Criterion: 14.6 +- 3.0
Avg. Flipped Nodes violating BC-Criterion: 28.27% +- 4.90%
Likelihood_change_l: [tensor(0.2642), tensor(0.2157), tensor(0.3636), tensor(0.2679), tensor(0.3023)]


In [142]:
seeds = [0, 1, 2, 3, 4]
n_s = [140, 500, 1000]
perform_experiment(n_s, seeds)


INFO:root:
Epoch    0: loss_train: 0.69189, loss_val: 0.69158, acc_train: 0.55072, acc_val: 0.56338
INFO:root:
Epoch  919: loss_train: 0.45602, loss_val: 0.52335, acc_train: 0.76812, acc_val: 0.76056


#Edges: 128 -> budget: 6


INFO:root:
Epoch    0: loss_train: 0.69228, loss_val: 0.69239, acc_train: 0.63768, acc_val: 0.59155


Accuracy Attack (trn): 0.7536231875419617
Accuracy model (trn): 0.8115942478179932, (val): 0.7323943376541138


INFO:root:
Epoch  227: loss_train: 0.48165, loss_val: 0.53197, acc_train: 0.76812, acc_val: 0.71831


...

#Edges: 1045 -> budget: 52
Accuracy Attack (trn): 0.7300000190734863
Accuracy model (trn): 0.8380000591278076, (val): 0.734000027179718


INFO:root:
Epoch    0: loss_train: 0.69274, loss_val: 0.69284, acc_train: 0.54000, acc_val: 0.53800
INFO:root:
Epoch  954: loss_train: 0.47423, loss_val: 0.54607, acc_train: 0.78800, acc_val: 0.73400


#Edges: 1017 -> budget: 51
Accuracy Attack (trn): 0.6859999895095825
Accuracy model (trn): 0.796000063419342, (val): 0.7300000190734863


INFO:root:
Epoch    0: loss_train: 0.69290, loss_val: 0.69320, acc_train: 0.49000, acc_val: 0.48000
INFO:root:
Epoch  347: loss_train: 0.50505, loss_val: 0.59935, acc_train: 0.76800, acc_val: 0.68800


#Edges: 998 -> budget: 50
Accuracy Attack (trn): 0.7139999866485596
Accuracy model (trn): 0.800000011920929, (val): 0.6840000152587891
Evaluting experiment with n=1000:
Note: #Edges: 998 -> budget: 50
Avg. Bayes Classification Error Trn: 85.80% +- 2.40%
Avg. Bayes Classification Error Val: 85.48% +- 1.38%
Avg. GNN Classification Error Trn: 80.52% +- 1.72%
Avg. GNN Classification Error Val: 70.52% +- 2.51%
Avg. Flipped Nodes: 51.6 +- 4.6
Avg. Flipped Nodes violating BC-Criterion: 6.8 +- 3.3
Avg. Flipped Nodes violating BC-Criterion: 12.90% +- 5.70%
Likelihood_change_l: [tensor(0.1698), tensor(0.0392), tensor(0.2000), tensor(0.1429), tensor(0.0930)]
